In [227]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = PROJECT  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud CAIP
TFVERSION = "1.14"                # TF version for CMLE to use
PROJ_NAME='miniproj'

In [228]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION
os.environ["PROJ_NAME"] = PROJ_NAME

In [229]:
FEATURE_LEN = 31
os.environ['FEATURE_LEN'] = str(FEATURE_LEN)

# 모델 실행

In [7]:
try:
    MODEL_NAME=input('모델 이름을 이력하세요:')
    if len(MODEL_NAME) < 1:
        MODEL_NAME='linear'
except ValueError:
    MODEL_NAME='linear'
os.environ["MODEL_NAME"] = MODEL_NAME
print(MODEL_NAME)

모델 이름을 이력하세요: dnnclass


dnnclass


In [222]:
%%bash
# MODEL_NAME=linear
DATADIR=$(pwd)
OUTDIR=$(pwd)/trained_${MODEL_NAME}/${PROJ_NAME}_v2
rm -rf $OUTDIR
gcloud ai-platform local train \
   --module-name=${PROJ_NAME}.trainer.task \
   --package-path=${PWD}/${PROJ_NAME}_v2.trainer \
   -- \
   --train_data_path="${DATADIR}/input/train_smote.csv" \
   --eval_data_path="${DATADIR}/input/valid.csv"  \
   --output_dir=${OUTDIR} \
   --model=${MODEL_NAME} \
   --train_steps=2000 \
   --feature_length=$FEATURE_LEN \
   --learning_rate=0.0001 \
   --eval_delay_secs=1 \
   --min_eval_frequency=10 \
   --stop_loss=1 \
   --nnsize='100,10'

Will use DNN size of [100, 10]
feature shape : Tensor("Squeeze:0", shape=(?, 29), dtype=float32)
feature shape : Tensor("Squeeze:0", shape=(?, 29), dtype=float32)
DNNClass MOdel : NNSIZE:[100, 10]
**************************************************
data_regressor : predictions shape Tensor("dense_2/Sigmoid:0", shape=(?, 1), dtype=float32)
compute_errors : features shape Tensor("Squeeze:0", shape=(?, 29), dtype=float32)
compute_errors : predictions shape Tensor("dense_2/Sigmoid:0", shape=(?, 1), dtype=float32)
feature shape : Tensor("Squeeze:0", shape=(?, 29), dtype=float32)
feature shape : Tensor("Squeeze:0", shape=(?, 29), dtype=float32)
DNNClass MOdel : NNSIZE:[100, 10]
**************************************************
data_regressor : predictions shape Tensor("dense_2/Sigmoid:0", shape=(?, 1), dtype=float32)
compute_errors : features shape Tensor("Squeeze:0", shape=(?, 29), dtype=float32)
compute_errors : predictions shape Tensor("dense_2/Sigmoid:0", shape=(?, 1), dtype=float32)
fea




INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/home/jupyter/google-asl-study/final_mini_project/youn/input/train_smote.csv', u'--eval_data_path=/home/jupyter/google-asl-study/final_mini_project/youn/input/valid.csv', u'--output_dir=/home/jupyter/google-asl-study/final_mini_project/youn/trained_dnnclass/miniproj_v2', u'--model=dnnclass', u'--train_steps=2000', u'--feature_length=31', u'--learning_rate=0.0001', u'--eval_delay_secs=1', u'--min_eval_frequency=10', u'--stop_loss=1', u'--nnsize=100,10'], u'job_name': u'miniproj.trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 10, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbd9b085d10>, '_model_dir': '/home/jupyter/google-asl-study/final_mini_project/youn/trained_d

# AI Platform JOB

In [225]:
!echo $BUCKET
!echo $PROJ_NAME

qwiklabs-gcp-ml-49b827b781ab
miniproj


In [231]:
!gsutil cp ./input/train_smote.csv gs://{BUCKET}/${PROJ_NAME}/input/train_smote.csv

Copying file://./input/train_smote.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][194.5 MiB/194.5 MiB]                                                
Operation completed over 1 objects/194.5 MiB.                                    


In [232]:
try:
    MODEL_NAME=input('모델 이름을 이력하세요:')
    if len(MODEL_NAME) < 1:
        MODEL_NAME='linear'
except ValueError:
    MODEL_NAME='linear'
os.environ["MODEL_NAME"] = MODEL_NAME
print(MODEL_NAME)

모델 이름을 이력하세요: dnnclass


dnnclass


In [233]:
%%bash
# MODEL_NAME=linear
DATADIR=$(pwd)
OUTDIR=gs://${BUCKET}/${PROJ_NAME}/${MODEL_NAME}_v2
JOBNAME=${PROJ_NAME}_${MODEL_NAME}_v2_$(date -u +%y%m%d_%H%M%S)

rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=trainer.task \
    --package-path=${PWD}/${PROJ_NAME}_v2/trainer \
    --job-dir=$OUTDIR \
    --scale-tier=PREMIUM_1 \
    --runtime-version=$TFVERSION \
    -- \
    --train_data_path="gs://${BUCKET}/${PROJ_NAME}/input/train_smote.csv" \
    --eval_data_path="gs://${BUCKET}/${PROJ_NAME}/input/valid.csv"  \
    --output_dir=${OUTDIR} \
    --model=${MODEL_NAME} \
    --train_steps=2000 \
    --feature_length=$FEATURE_LEN \
    --learning_rate=0.0001 \
    --eval_delay_secs=1 \
    --min_eval_frequency=10 \
    --stop_loss=1 \
    --nnsize='100,10'

jobId: miniproj_dnnclass_v2_191211_134656
state: QUEUED


Job [miniproj_dnnclass_v2_191211_134656] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe miniproj_dnnclass_v2_191211_134656

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs miniproj_dnnclass_v2_191211_134656


# yamul파일 사용

In [29]:
%%writefile hyperparam.yaml
trainingInput:
    scaleTier: PREMIUM_1
    hyperparameters:
        hyperparameterMetricTag: RMSE
        goal: MINIMIZE
        maxTrials: 4
        maxParallelTrials: 2
        enableTrialEarlyStopping: True
        params:
        - parameterName: nnsize
          type: CATEGORICAL
          categoricalValues:
          - 30,10
          - 100,10
          - 30,100
          - 30,10,30        

Overwriting hyperparam.yaml


In [30]:
%%bash
# MODEL_NAME=linear
DATADIR=$(pwd)
OUTDIR=gs://${BUCKET}/${PROJ_NAME}/${MODEL_NAME}_HParam
JOBNAME=${PROJ_NAME}_${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)

rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=trainer.task \
    --package-path=${PWD}/${PROJ_NAME}/trainer \
    --job-dir=$OUTDIR \
    --scale-tier=PREMIUM_1 \
    --runtime-version=$TFVERSION \
    --config=hyperparam.yaml \
    -- \
    --train_data_path="gs://${BUCKET}/${PROJ_NAME}/input/train_smote.csv" \
    --eval_data_path="gs://${BUCKET}/${PROJ_NAME}/input/valid.csv"  \
    --output_dir=${OUTDIR} \
    --model=${MODEL_NAME} \
    --train_steps=2000 \
    --feature_length=$FEATURE_LEN \
    --learning_rate=0.0001 \
    --eval_delay_secs=1 \
    --min_eval_frequency=10 \
    --stop_loss=1 \
    --nnsize='30,10'

jobId: miniproj_dnnclass_191210_071526
state: QUEUED


Job [miniproj_dnnclass_191210_071526] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe miniproj_dnnclass_191210_071526

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs miniproj_dnnclass_191210_071526


# 코드 생성

In [219]:
%%bash
mkdir -p ${PROJ_NAME}_v2/trainer
touch ${PROJ_NAME}_v2/__init__.py
touch ${PROJ_NAME}_v2/trainer/__init__.py

In [220]:
%%writefile {PROJ_NAME}_v2/trainer/task.py
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
        "--nnsize",
        help="Hidden layer sizes to use for DNN feature columns -- provide \
        space-separated layers",
        type=str,
        default='128,32,4'
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--feature_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )
  parser.add_argument(
      '--stop_loss',
      help='Value of Early Stopping Loss',
      default=1,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')
  model.BATCH_SIZE = hparams['train_batch_size']
  model.NNSIZE = list(map(int, hparams.pop("nnsize").split(",")))
  print ("Will use DNN size of {}".format(model.NNSIZE))

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Writing miniproj_v2/trainer/task.py


In [221]:
%%writefile {PROJ_NAME}_v2/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)


N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
FEATURE_LEN = None
DEFAULTS = None
N_INPUTS = None
BATCH_SIZE = 512
NNSIZE = [64, 16, 4]

CSV_COLUMNS = 'Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class'.split(',')
FEATURES_COLUMNS = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL_COLUMN = 'Class'
CSV_DEFAULTS = [[0.0] for i in range(31)]



def init(hparams):
    global FEATURE_LEN, DEFAULTS, N_INPUTS
    FEATURE_LEN = hparams['feature_length']
    DEFAULTS = [[0.0] for x in range(0, FEATURE_LEN)]
    N_INPUTS = FEATURE_LEN - N_OUTPUTS

def feature_merge(features):
    temp_V01 = tf.expand_dims(features["V1"] , -1)
    temp_V02 = tf.expand_dims(features["V2"] , -1)
    temp_V03 = tf.expand_dims(features["V3"] , -1)
    temp_V04 = tf.expand_dims(features["V4"] , -1)
    temp_V05 = tf.expand_dims(features["V5"] , -1)
    temp_V06 = tf.expand_dims(features["V6"] , -1)
    temp_V07 = tf.expand_dims(features["V7"] , -1)
    temp_V08 = tf.expand_dims(features["V8"] , -1)
    temp_V09 = tf.expand_dims(features["V9"] , -1)
    temp_V10 = tf.expand_dims(features["V10"], -1)
    temp_V11 = tf.expand_dims(features["V11"], -1)
    temp_V12 = tf.expand_dims(features["V12"], -1)
    temp_V13 = tf.expand_dims(features["V13"], -1)
    temp_V14 = tf.expand_dims(features["V14"], -1)
    temp_V15 = tf.expand_dims(features["V15"], -1)
    temp_V16 = tf.expand_dims(features["V16"], -1)
    temp_V17 = tf.expand_dims(features["V17"], -1)
    temp_V18 = tf.expand_dims(features["V18"], -1)
    temp_V19 = tf.expand_dims(features["V19"], -1)
    temp_V20 = tf.expand_dims(features["V20"], -1)
    temp_V21 = tf.expand_dims(features["V21"], -1)
    temp_V22 = tf.expand_dims(features["V22"], -1)
    temp_V23 = tf.expand_dims(features["V23"], -1)
    temp_V24 = tf.expand_dims(features["V24"], -1)
    temp_V25 = tf.expand_dims(features["V25"], -1)
    temp_V26 = tf.expand_dims(features["V26"], -1)
    temp_V27 = tf.expand_dims(features["V27"], -1)
    temp_V28 = tf.expand_dims(features["V28"], -1)
    temp_Amount = tf.expand_dims(features["Amount"],-1)
    features = tf.stack([temp_V01, temp_V02, temp_V03, temp_V04, temp_V05,
                  temp_V06, temp_V07, temp_V08, temp_V09, temp_V10,
                  temp_V11, temp_V12, temp_V13, temp_V14, temp_V15,
                  temp_V16, temp_V17, temp_V18, temp_V19, temp_V10,
                  temp_V21, temp_V22, temp_V23, temp_V24, temp_V25,
                  temp_V26, temp_V27, temp_V28, temp_Amount
                 ], axis=1)
    features = tf.squeeze(features, -1)
    print('feature shape : {}'.format(features))

    return features

def linear_model(features, mode, params):
    X = features
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features
    print('NNSIZE:{}'.format(NNSIZE))
    for unit_num in NNSIZE:
        X = tf.layers.dense(X, units=unit_num, activation=tf.nn.relu)
    predictions = tf.layers.dense(X, 1, activation=None)  # linear output: regression
    return predictions


def dnnclass_model(features, mode, params):
    
    print('feature shape : {}'.format(features))
    print('DNNClass MOdel : NNSIZE:{}'.format(NNSIZE))
    print('*'*50)
    
    X = features
    for unit_num in NNSIZE:
        X = tf.layers.dense(X, units=unit_num, activation=tf.nn.relu)
    predictions = tf.layers.dense(X, 1, activation=tf.nn.sigmoid)  # logistic output
    return predictions


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            fields = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)
            features = dict(zip(CSV_COLUMNS, fields))
            features.pop("Time")
            label = features.pop(LABEL_COLUMN)
            return features, label        
        
        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        
        dataset = dataset.skip(1).map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        key: tf.placeholder(tf.float32, [None]) for key in FEATURES_COLUMNS
    }
    features = feature_placeholders
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    print('compute_errors : features shape {}'.format(features))

    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    #if predictions.shape[1] == 1:
    #AttributeError: 'NoneType' object has no attribute 'shape'
    print('compute_errors : predictions shape {}'.format(predictions))
    
    
    if predictions.shape[1] == 1:
        #loss = tf.losses.mean_squared_error(labels, predictions)
        loss = tf.losses.sigmoid_cross_entropy(labels, predictions)
        #loss = tf.Print(loss, [tf.shape(loss), loss], summarize = -1)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        recall = tf.metrics.recall(labels, predictions)
        precision = tf.metrics.precision(labels, predictions)
        True_positives = tf.metrics.true_positives(labels, predictions)
        True_negatibes = tf.metrics.true_negatives(labels, predictions)
        False_negatibes = tf.metrics.false_negatives(labels, predictions)
        False_positives = tf.metrics.false_positives(labels, predictions)
        return loss, rmse, recall, precision, True_positives, True_negatibes, False_negatibes, False_positives
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features, labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        #loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        loss = tf.losses.sigmoid_cross_entropy(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        recall = tf.metrics.recall(labels, lastPred)
        precision = tf.metrics.precision(labels, predictions)
        True_positives = tf.metrics.true_positives(labels, predictions)
        True_negatibes = tf.metrics.true_negatives(labels, predictions)
        False_negatibes = tf.metrics.false_negatives(labels, predictions)
        False_positives = tf.metrics.false_positives(labels, predictions)
        return loss, rmse, recall, precision, True_positives, True_negatibes, False_negatibes, False_positives

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def data_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'dnnclass': dnnclass_model}
    model_function = model_functions[params['model']]
    
    features = feature_merge(features)
    #print('2>>>>>>> {}'.format(features))

    
    predictions = model_function(features, mode, params)
    #predictions = tf.Print(predictions, [tf.shape(predictions), predictions], summarize=-1)
    
    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        print('data_regressor : predictions shape {}'.format(predictions))
        loss, rmse, recall, precision, True_positives, True_negatibes, False_negatibes, False_positives = compute_errors(features, labels, predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
            "Recall": recall,
            "Precision": precision,
            "True_positives": True_positives,
            "True_negatibes": True_negatibes,
            "False_negatibes": False_negatibes,
            "False_positives": False_positives,
        }

        
    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    
    estimator = tf.estimator.Estimator(model_fn=data_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           tf_random_seed = 1,
                                           save_checkpoints_secs=
                                           hparams['min_eval_frequency']),
                                       model_dir=output_dir)
   
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'],
                                        hooks=[tf.estimator.experimental.stop_if_no_decrease_hook(estimator, "loss", hparams['stop_loss'])])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      start_delay_secs=hparams['eval_delay_secs'],
                                      throttle_secs=hparams['min_eval_frequency'])
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


Writing miniproj_v2/trainer/model.py
